In [2]:
from aerobot.utils import FEATURE_TYPES, DATA_PATH, MODELS_PATH, save_hdf, RNA16S_PATH
from aerobot.models import NonlinearClassifier
from aerobot.dataset import FeatureDataset
from aerobot import ncbi
import os 
import pandas as pd
import numpy as np
import wget
import zipfile
import warnings
import tables
import shutil

# Ignore some annoying warnings triggered when saving HDF files.
warnings.filterwarnings('ignore', category=pd.io.pytables.PerformanceWarning)
warnings.filterwarnings('ignore', category=tables.NaturalNameWarning)

%load_ext autoreload 
%autoreload 2

In [4]:
rna16s_training_dataset = FeatureDataset(os.path.join(RNA16S_PATH, 'training_dataset.h5'), feature_type='embedding_rna16s')
rna16s_testing_dataset = FeatureDataset(os.path.join(RNA16S_PATH, 'testing_dataset.h5'), feature_type='embedding_rna16s')
rna16s_validation_dataset = FeatureDataset(os.path.join(RNA16S_PATH, 'validation_dataset.h5'), feature_type='embedding_rna16s')

print('Size of RNA 16S training dataset:', len(rna16s_training_dataset))
print('Size of RNA 16S testing dataset:', len(rna16s_testing_dataset))
print('Size of RNA 16S validation dataset:', len(rna16s_validation_dataset))
print()
print('Total number of 16S RNA sequences:', len(rna16s_training_dataset) + len(rna16s_testing_dataset) + len(rna16s_validation_dataset))

Size of RNA 16S training dataset: 693
Size of RNA 16S testing dataset: 188
Size of RNA 16S validation dataset: 150

Total number of 16S RNA sequences: 1031


In [52]:
# Get the list of suppressed genomes from the NCBI FTP site. 
# These genomes did not meet these standards: https://www.ncbi.nlm.nih.gov/datasets/docs/v2/policies-annotation/genome-processing/genome_notes/ 
wget.download('ftp://ftp.ncbi.nlm.nih.gov/genomes/ASSEMBLY_REPORTS/assembly_summary_refseq_historical.txt', os.path.join(DATA_PATH, 'suppressed_genomes.tsv'))
suppressed_genomes_df = pd.read_csv(os.path.join(DATA_PATH, 'suppressed_genomes.tsv'), delimiter='\t', skiprows=1, index_col=0)
suppressed_genomes_df.index.name = 'id'
suppressed_genomes_df = suppressed_genomes_df[['version_status', 'organism_name', 'annotation_provider', 'annotation_date']]
suppressed_genomes_df.to_csv(os.path.join(DATA_PATH, 'suppressed_genomes.csv'))

'/home/prichter/Documents/aerobot/aerobot/../data/suppressed_genomes.tsv'

In [8]:
# Getting tired of contending with all the filename mapping, so just going to go ahead and adjust the source data.

old_madin_data = pd.HDFStore(os.path.join(DATA_PATH, 'original', 'madin.h5'))
new_madin_data = dict()

# Split the things which were previously stored as metadata into their individual features. 
old_metadata = old_madin_data['AF'].rename(columns={'oxygen_genes':'number_of_oxygen_genes'}).drop_duplicates()
old_metadata['percent_oxygen_genes'] = old_metadata['number_of_oxygen_genes'] / old_metadata['number_of_genes']
new_madin_data['number_of_genes'] = old_metadata[['number_of_genes']] 
new_madin_data['number_of_oxygen_genes'] = old_metadata[['number_of_oxygen_genes']] 
new_madin_data['percent_oxygen_genes'] = old_metadata[['percent_oxygen_genes']] 

# Store what used to be called "labels" under "metadata." Drop some of the long, annoying columns.
new_madin_data['metadata'] = old_madin_data['labels'].drop(columns=['annotation_file', 'embedding_file']).drop_duplicates()
new_madin_data['ko'] = old_madin_data['KO']
new_madin_data['embedding_genome'] = old_madin_data['WGE']
new_madin_data['embedding_oxygen_genes'] = old_madin_data['OGSE']

exclude_feature_types = ['ko_terminal_oxidase_genes', 'chemical', 'embedding_rna16s']
for feature_type in FEATURE_TYPES:
    if (feature_type not in new_madin_data) and (feature_type not in exclude_feature_types):
        new_madin_data[feature_type] = old_madin_data[feature_type]

save_hdf(new_madin_data, os.path.join(DATA_PATH, 'madin_datasets.h5'))

In [ ]:
# Also going to put the Jablonska data into an HDF file for consistency. 
new_jablonska_data = dict()

# Split the things which were previously stored as metadata into their individual features. 
old_metadata =  pd.read_csv(os.path.join(DATA_PATH, 'original', f'jablonska_metadata.csv'), index_col=0)
old_metadata = old_metadata.rename(columns={'oxygen_genes':'number_of_oxygen_genes'}).drop_duplicates()
old_metadata = old_metadata.set_index('genome')
old_metadata['percent_oxygen_genes'] = old_metadata['number_of_oxygen_genes'] / old_metadata['number_of_genes']
old_metadata = old_metadata.rename({'Oder':'Order'})

new_jablonska_data['number_of_genes'] = old_metadata[['number_of_genes']] 
new_jablonska_data['number_of_oxygen_genes'] = old_metadata[['number_of_oxygen_genes']] 
new_jablonska_data['percent_oxygen_genes'] = old_metadata[['percent_oxygen_genes']] 

# Store what used to be called "labels" under "metadata." Drop some of the long, annoying columns.
new_metadata = pd.read_csv(os.path.join(DATA_PATH, 'original', f'jablonska_labels.csv'), index_col=0).drop(columns=['annotation_file', 'embedding_file']).drop_duplicates()
new_metadata = new_metadata.rename(columns={'Oder':'Order'})
new_jablonska_data['metadata'] = new_metadata

exclude_feature_types = ['ko_terminal_oxidase_genes', 'chemical', 'embedding_rna16s']
for feature_type in FEATURE_TYPES:
    if (feature_type not in new_jablonska_data) and (feature_type not in exclude_feature_types):
        new_jablonska_data[feature_type] = pd.read_csv(os.path.join(DATA_PATH, 'original', f'jablonska_{feature_type}.csv'), index_col=0)

save_hdf(new_jablonska_data, os.path.join(DATA_PATH, 'jablonska_datasets.h5'))

Curious about how much the annotated KO groups in the Jablonska and Madin datasets overlapped. I am running into a bug due to the fact that one (or both) of the datasets have no proteins annotated with some of the KO groups in the list of terminal oxidase KO groups. I am trying to figure out if it is better to fill in zeros or drop the missing KO groups.

If both datasets are missing the same KO groups, I will want to drop the KO groups which are not present. If I fill in with zeros, then the model weights corresponding to those input features will not be trained. If input data which *does* contain those KO groups is used as input to the trained model, then it could adversely effect predictions.

I first want to make sure that the columns in the Jablonska and Madin KO data are the same. 

In [92]:
testing_genomes = pd.read_hdf(os.path.join(DATA_PATH, 'testing_datasets.h5'), key='metadata').index


GCA_000010085.1 GCA_000069185.1 GCA_000093085.1 GCA_002339675.1 GCA_002363385.1 GCA_002386405.1 GCA_003165155.1 GCA_003241595.1 GCA_003248565.1 GCA_003248865.1 GCA_003447235.1 GCA_003483565.1 GCA_003510825.1 GCA_003694355.1 GCA_004298145.1 GCA_004554205.1 GCA_004796465.1 GCA_004799495.1 GCA_005779435.1 GCA_009886055.1 GCA_009886125.1 GCA_011524565.1 GCA_012270465.1 GCA_012523755.1 GCA_012837295.1 GCA_013361785.1 GCA_014376935.1 GCA_014801735.1 GCA_014801785.1 GCA_014801815.1 GCA_014801865.1 GCA_014801985.1 GCA_014802105.1 GCA_014802425.1 GCA_014803725.1 GCA_014804865.1 GCA_015058375.1 GCA_015255025.1 GCA_016177775.1 GCA_016208105.1 GCA_016281815.1 GCA_016284635.1 GCA_016286585.1 GCA_016287755.1 GCA_016291915.1 GCA_016703745.1 GCA_016710135.1 GCA_016841205.1 GCA_017304655.1 GCA_017402445.1 GCA_017423565.1 GCA_017435645.1 GCA_017449675.1 GCA_017458005.1 GCA_017459445.1 GCA_017460515.1 GCA_017467565.1 GCA_017467705.1 GCA_017469955.1 GCA_017479675.1 GCA_017483185.1 GCA_017538195.1 GCA_0175

Seems like a best call to use the union of the columns in each dataset. This should maximize the amount of information, while ensuring that all weights are actually updated during model training.

In [ ]:
# Code I ran to clean up the Westoby data and add taxonomy. 

metadata = pd.read_csv(os.path.join(DATA_PATH, 'Mark_Westoby_Organism_Metadata_Export_02152018.tsv'), sep='\t', index_col=0).GENBANK_16S_ID
print(metadata)
metadata = metadata[['GENBANK_16S_ID','OXYGEN_REQUIREMENT', 'NCBI_TAXONOMY_ID']] # Grab the relevant columns from the metadata DataFrame.

metadata = metadata[~metadata['GENBANK_16S_ID'].str.contains('(null)', regex=False)]
metadata = metadata[~metadata['GENBANK_16S_ID'].str.contains('(null)', regex=False)]
metadata.columns = ['id', 'physiology', 'ncbi_taxonomy_id']

# Some of the IDs have multiple GenBank accessions listed for the same organism (comma-separated)
# I assume these are duplicates, and just used the first one. 
metadata['id'] = [id_.split(',')[0] for id_ in metadata['id']] 
# Other IDs seem to have extra information appended to the end, following a colon. I assume I can remove this. 
metadata['id'] = [id_.split(':')[0] for id_ in metadata['id']]

label_map = {'Anaerobe':'anaerobe', 'Aerobe':'aerobe', 'Facultative':'facultative', 'Obligate anaerobe':'anaerobe', 'Obligate aerobe':'aerobe','Facultative anaerobe':'facultative'}
metadata = metadata[metadata.physiology.isin(label_map)] # Filter for organisms with an oxygen label is in the map. 
metadata.physiology = metadata.physiology.replace(label_map)
metadata = metadata.drop_duplicates('id')
metadata = metadata.set_index('id')

# TODO: How should I handle the no-rank ones?
taxonomy = entrez.download_taxonomy(metadata.ncbi_taxonomy_id)
cols = metadata.columns.union(taxonomy.columns)
metadata = metadata.merge(taxonomy, left_on=['ncbi_taxonomy_id'], right_index=True, how='inner')[cols]
metadata = metadata[~metadata.index.duplicated(keep='first')]
metadata.to_csv(os.path.join(DATA_PATH, 'rna16s_metadata.csv'))

In [20]:
# I might need to install gslm, here: https://github.com/ramanathanlab/genslm/blob/main/setup.cfg 
# Needed to install a Rust compiler before installing this. with this command: curl --proto '=https' --tlsv1.2 https://sh.rustup.rs -sSf | sh
# After installing Rust, I also needed to run apt install pkg-config
# Still didn't work, so I tried installing libssl-dev using apt. 
# Still didn't work, so I tried downgrading Python to 3.7. That finally worked. Not sure why, something with bugs when trying to install h5py and tokenizers. 
# Unfortunately, when I did this, I need to reinstall a ton of stuff. 

In [132]:
# Seems to be a lot of downloaded genomes... what is the comparative size of the training and testing set?
test_dataset = FeatureDataset(os.path.join(DATA_PATH, 'testing_datasets.h5'), feature_type='aa_1mer')
train_dataset = FeatureDataset(os.path.join(DATA_PATH, 'training_datasets.h5'), feature_type='percent_oxygen_genes')
val_dataset = FeatureDataset(os.path.join(DATA_PATH, 'validation_datasets.h5'), feature_type='aa_1mer')

print('Size of testing dataset:', len(test_dataset))
print('Size of training dataset:', len(train_dataset))
print('Size of validation dataset:', len(val_dataset))

Size of testing dataset: 587
Size of training dataset: 2084
Size of validation dataset: 465


In [7]:
contigs_datasets = FeatureDataset(os.path.join(DATA_PATH, 'rna16s', 'testing_dataset.h5'), feature_type='aa_1mer')
contigs_datasets.metadata

,genome_id,contig_size,Class,physiology,aa_1mer_prediction,aa_2mer_prediction,aa_3mer_prediction
id,,,,,,,
GCF_000425825_2000_0,GCF_000425825,2000,Bacilli,Facultative,facultative,facultative,facultative
GCF_000425825_2000_1,GCF_000425825,2000,Bacilli,Facultative,facultative,facultative,facultative
GCF_000425825_2000_2,GCF_000425825,2000,Bacilli,Facultative,facultative,facultative,facultative
GCF_000425825_2000_3,GCF_000425825,2000,Bacilli,Facultative,facultative,facultative,facultative
GCF_000425825_2000_4,GCF_000425825,2000,Bacilli,Facultative,facultative,facultative,facultative
...,...,...,...,...,...,...,...
GCF_000008665_40000_45,GCF_000008665,40000,Archaeoglobi,Anaerobe,anaerobe,anaerobe,anaerobe
GCF_000008665_40000_46,GCF_000008665,40000,Archaeoglobi,Anaerobe,anaerobe,anaerobe,anaerobe
GCF_000008665_40000_47,GCF_000008665,40000,Archaeoglobi,Anaerobe,anaerobe,anaerobe,anaerobe


In [6]:
contigs_datasets.metadata.drop_duplicates()

,genome_id,contig_size,Class,physiology,aa_1mer_prediction,aa_2mer_prediction,aa_3mer_prediction
id,,,,,,,
GCF_000425825_2000_0,GCF_000425825,2000,Bacilli,Facultative,facultative,facultative,facultative
GCF_000425825_5000_0,GCF_000425825,5000,Bacilli,Facultative,facultative,facultative,facultative
GCF_000425825_7000_0,GCF_000425825,7000,Bacilli,Facultative,facultative,facultative,facultative
GCF_000425825_10000_0,GCF_000425825,10000,Bacilli,Facultative,facultative,facultative,facultative
GCF_000425825_20000_0,GCF_000425825,20000,Bacilli,Facultative,facultative,facultative,facultative
...,...,...,...,...,...,...,...
GCF_000008665_7000_0,GCF_000008665,7000,Archaeoglobi,Anaerobe,anaerobe,anaerobe,anaerobe
GCF_000008665_10000_0,GCF_000008665,10000,Archaeoglobi,Anaerobe,anaerobe,anaerobe,anaerobe
GCF_000008665_20000_0,GCF_000008665,20000,Archaeoglobi,Anaerobe,anaerobe,anaerobe,anaerobe


In [8]:
store = pd.HDFStore(os.path.join(DATA_PATH, 'rna16s', 'testing_dataset.h5'))
# store.get_node('embedding.rna16s')._f_rename('embedding_rna16s')
store.get_node('labels')._f_rename('metadata')
store.close()

store = pd.HDFStore(os.path.join(DATA_PATH, 'rna16s', 'training_dataset.h5'))
# store.get_node('embedding.rna16s')._f_rename('embedding_rna16s')
store.get_node('labels')._f_rename('metadata')
store.close()

store = pd.HDFStore(os.path.join(DATA_PATH, 'rna16s', 'validation_dataset.h5'))
# store.get_node('embedding.rna16s')._f_rename('embedding_rna16s')
store.get_node('labels')._f_rename('metadata')
store.close()

In [4]:
df = pd.read_hdf(os.path.join(DATA_PATH, 'validation_datasets.h5'), key='chemical')
df.columns

Index(['gc_content', 'number_of_genes', 'aa_mean_num_carbon',
       'aa_mean_num_oxygen', 'aa_mean_num_nitrogen', 'aa_mean_num_sulfur',
       'cds_mean_num_carbon', 'cds_mean_num_oxygen', 'cds_mean_num_nitrogen'],
      dtype='object')